### Задание по программированию: Выбор числа соседей

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import scale

#### 1. Загрузите выборку Wine по адресу https://archive.ics.uci.edu/ml/machinelearning-databases/wine/wine.data

In [2]:
header = ["class", "alcohol", "malic_acid", "ash", "ash_alcalinity", "magnesium", "total_phenols", "flavanoids", 
    "nonflavanoid_phenols", "proanthocyanins", "color_intensity", "hue", "od280", "proline"
]
data = pd.read_csv("./wine.data", names = header)

#### 2. Извлеките из данных признаки и классы. Класс записан в первом столбце (три варианта), признаки — в столбцах со второго по последний. 

In [6]:
X = data.iloc[:, 1:]
y = data.iloc[:, 0]

In [3]:
data.head()

,class,alcohol,malic_acid,ash,ash_alcalinity,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280,proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


#### 3. Оценку качества необходимо провести методом кросс-валидации по 5 блокам (5-fold). Создайте генератор разбиений, который перемешивает выборку перед формированием блоков (shuffle=True). Для воспроизводимости результата, создавайте генератор KFold с фиксированным параметром random_state=42. В качестве меры качества используйте долю верных ответов (accuracy).

In [63]:
def calculate_classification_score(features, answers):
    fold = KFold(shuffle = True, random_state = 42, n_splits = 5)
    model = KNeighborsClassifier(n_neighbors = 1)
    
    grid_params = {"n_neighbors":np.arange(1, 50)}

    grid_search = GridSearchCV(model, refit = True, param_grid = grid_params, cv = fold, iid = False)
    grid_search.fit(features, answers)
    
    best_estimator = grid_search.best_estimator_
    
    scores = cross_val_score(estimator = best_estimator, cv = fold, scoring = "accuracy", X = features, y = answers)
    return {"mean_scores": np.mean(scores), "best_params": grid_search.best_params_}

#### 4. Найдите точность классификации на кросс-валидации для метода k ближайших соседей (sklearn.neighbors.KNeighborsClassifier), при k от 1 до 50. При каком k получилось оптимальное качество? Чему оно равно (число в интервале от 0 до 1)?

In [60]:
calculate_classification_score(X, y)

{'best_params': {'n_neighbors': 1}, 'mean_scores': 0.7304761904761905}

#### 5. Произведите масштабирование признаков с помощью функции sklearn.preprocessing.scale. Снова найдите оптимальное k на кроссвалидации.

In [61]:
X_scaled = scale(X.values)

In [62]:
calculate_classification_score(X_scaled, y)

{'best_params': {'n_neighbors': 29}, 'mean_scores': 0.9776190476190475}